In [ ]:
%matplotlib inline

cur_dir = os.path.dirname('__file__')

train = pd.read_csv(os.path.join(cur_dir, "data", "train.csv"))
test = pd.read_csv(os.path.join(cur_dir, "data", "test.csv"))

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns